In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import collections

Using TensorFlow backend.


In [3]:
with open("flickr30k_images/results.txt",'r') as f:
    info = f.read()
info = info.split('\n')
information = []
for i in range(1,len(info)-1):
    information.append(info[i].split('|'))

In [5]:
descriptions = {}
for i in information:
    if descriptions.get(i[0]) is None:
        descriptions[i[0]] = []
    descriptions[i[0]].append(i[2])

In [7]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[^a-z]+"," ",sentence)
    sentence = sentence.split()
    
    sentence = [s for s in sentence if len(s)>1]
    sentence = ' '.join(sentence)
    return sentence

In [8]:
for key,caption_list in descriptions.items():
    for i in range(len(caption_list)):
        caption_list[i] = clean_text(caption_list[i])

In [10]:
with open("descriptions.txt",'w') as f:
    f.write(str(descriptions))

In [11]:
train_descriptions = {}

for img_id in descriptions.keys():
    train_descriptions[img_id] = []
    for cap in descriptions[img_id]:
        cap_to_append = "<s> "+cap+" <e>"
        train_descriptions[img_id].append(cap_to_append)

In [15]:
with open('train_descriptions.pkl', 'wb') as f:
        pickle.dump(train_descriptions, f, pickle.HIGHEST_PROTOCOL)

In [18]:
train_descriptions = None
with open('train_descriptions.pkl', 'rb') as f:
        train_descriptions = pickle.load(f)

In [23]:
len(train_descriptions)

31783

In [24]:
total_words = []

for key in descriptions.keys():
    [total_words.append(i) for des in descriptions[key] for i in des.split()]

In [25]:
len(total_words)

1679080

In [26]:
counter = collections.Counter(total_words)
freq = dict(counter)
print(len(freq))

threshold=10
shortlist = [x for x in freq.keys() if freq[x]>threshold]

print(len(shortlist))

18054
5118


In [29]:
word_to_index = {}
index_to_word = {}

for i,word in enumerate(shortlist):
    word_to_index[word] = i+1
    index_to_word[i+1] = word

In [36]:
word_to_index["<s>"] = 5119
index_to_word[5119] = "<s>"

word_to_index["<e>"] = 5120
index_to_word[5120] = "<e>"

In [37]:
vocab_size = len(word_to_index)+1
vocab_size

5121

In [38]:
with open('word_to_index.pkl', 'wb') as f:
        pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)

In [39]:
with open('index_to_word.pkl', 'wb') as f:
        pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)

In [40]:
f = open("glove.6B.50d.txt")

embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    word_embedding = np.array(values[1:],dtype='float')
    embedding_index[word] = word_embedding

f.close()

In [41]:
def get_embedding_matrix():
    emb_dim = 50
    matrix = np.zeros((vocab_size,emb_dim))
    for word,idx in word_to_index.items():
        embedding_vector = embedding_index.get(word)
        
        if embedding_vector is not None:
            matrix[idx] = embedding_vector
    return matrix

In [53]:
embedding_matrix = get_embedding_matrix()
embedding_matrix.shape

(5121, 50)

In [56]:
with open('embedding_matrix.pkl', 'wb') as f:
        pickle.dump(embedding_matrix, f, pickle.HIGHEST_PROTOCOL)

In [68]:
encoding_train_features = None
with open("encoding_train_features.pkl",'rb') as f:
    encoding_train_features = pickle.load(f)